## Tarea Clase 19: Opciones Asiáticas

### Alumno: Esteban Navarrete Garrido

In [1]:
#importar los paquetes que se van a usar
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
%matplotlib inline
#algunas opciones para Pandas
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)

In [6]:
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    T = 1
    nu = mu-(sigma**2)/2
    
    DeltaT = T/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    
    #for i in range(NbStep):
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    LogSt = np.cumsum(concat,axis=1)
    St = np.exp(LogSt)
    t = np.arange(0,1,DeltaT)

    return St.T,t

def calc_daily_ret(closes):
    return np.log(closes/closes.shift(1)).iloc[1:]

## CALL

In [3]:
NbTraj = [100,1000,10000,50000]
NbStep = [10,50,100]

S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años

Call = np.zeros([len(NbTraj),len(NbStep)])
intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
i = 0    # controla las filas
j = 0    # controla las columnas

for tray in NbTraj:
    j = 0
    for nescen in NbStep:
        St,t = BSprices(r,sigma,S0,tray,nescen)
        prices = pd.DataFrame(St,index=t)
        Average_t = prices.expanding().mean()
        strike = pd.DataFrame(K*np.ones([nescen,tray]), index=t)
        call = pd.DataFrame({'Prima':np.exp(-r*T) \
             *np.fmax(Average_t-strike,np.zeros([nescen,tray])).mean(axis=1)}, index=t)
        Call[i,j]= call.iloc[-1]
        # intervalos de confianza
        confianza = 0.95
        sigma_est = prices.iloc[-1].sem()
        mean_est = call.iloc[-1].Prima
#        i1 = st.t.interval(confianza,nescen-1, loc=mean_est, scale=sigma_est)
        i2 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
        intervalos.append(np.asarray(i2))
        j+=1
    i+=1

intervalos = np.reshape(np.asarray(intervalos),[3,6])
filas = ['Nbtray = %i' %i for i in NbTraj]
col = ['NbStep = %i' %i for i in NbStep]+['int %i' %j for j in range(6)]
df = pd.DataFrame(index=filas,columns=col)
df.loc[0:4,0:3] = Call
df.loc[0:,3:] = intervalos
df

,NbStep = 10,NbStep = 50,NbStep = 100,...,int 3,int 4,int 5
Nbtray = 1000,6.57,6.79,7.39,...,8.16,6.01,8.77
Nbtray = 5000,6.49,6.81,6.72,...,7.42,6.11,7.32
Nbtray = 10000,6.43,6.92,7,...,7.35,6.57,7.42


## PUT

In [10]:
NbTraj = [100,1000,10000,50000]
NbStep = [10,50,100]

S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años

Put = np.zeros([len(NbTraj),len(NbStep)])
intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
i = 0    # controla las filas
j = 0    # controla las columnas

for tray in NbTraj:
    j = 0
    for nescen in NbStep:
        St,t = BSprices(r,sigma,S0,tray,nescen)
        prices = pd.DataFrame(St,index=t)
        Average_t = prices.expanding().mean()
        strike = pd.DataFrame(K*np.ones([nescen,tray]), index=t)
        put = pd.DataFrame({'Prima':np.exp(-r*T) \
             *np.fmax(strike-Average_t,np.zeros([nescen,tray])).mean(axis=1)}, index=t)
        Put[i,j]= put.iloc[-1]
        # intervalos de confianza
        confianza = 0.95
        sigma_est = prices.iloc[-1].sem()
        mean_est = put.iloc[-1].Prima
#        i1 = st.t.interval(confianza,nescen-1, loc=mean_est, scale=sigma_est)
        i2 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
        intervalos.append(np.asarray(i2))
        j+=1
    i+=1

intervalos = np.reshape(np.asarray(intervalos),[4,6])
filas = ['Nbtray = %i' %i for i in NbTraj]
col = ['NbStep = %i' %i for i in NbStep]+['int %i' %j for j in range(6)]
df = pd.DataFrame(index=filas,columns=col)
df.loc[0:4,0:3] = Put
df.loc[0:,3:] = intervalos
df

,NbStep = 10,NbStep = 50,NbStep = 100,...,int 3,int 4,int 5
Nbtray = 100,2.51,2.38,2.9,...,6.48,-0.618,6.41
Nbtray = 1000,2.04,2.21,2.6,...,3.57,1.24,3.96
Nbtray = 10000,2.19,2.33,2.3,...,2.77,1.87,2.73
Nbtray = 50000,2.23,2.32,2.33,...,2.51,2.13,2.52
